# Re-grid mirrored CMIP6 data on ACDN

Use this notebook to re-grid all of the mirrored CMIP6 data to a common grid. We will use the grid that is common between the majority of the CESM2, CESM2-WACCM, NorESM2, and TaiESM1 data. 

This notebook sets up the `.slurm` files to then be submitted with `sbatch`. Each sbatch file consists of submitting a single job to process the files listed in a single batch file, so there is a single `.slurm` file for each batch file created with `generate_batch_files.py`.

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import xesmf as xe

from config import *
from slurm import *

# ignore serializationWarnings from xarray for datasets with multiple FillValues
import warnings

warnings.filterwarnings("ignore", category=xr.SerializationWarning)


Iterate over the batch files and create slurm `sbatch` scripts to regrid all filepaths contained in those files (one job for each file).

Change the `no_clobber` option below to `True` to avoid overwriting any regridded files that already exist. 

In [2]:
sbatch_fps = []
sbatch_dir = slurm_dir.joinpath("regrid")
sbatch_dir.mkdir(exist_ok=True)
for fp in regrid_batch_dir.glob("*.txt"):
    sbatch_str = fp.name.split("batch_")[1].split(".txt")[0]
    sbatch_fp = sbatch_dir.joinpath(f"regrid_{sbatch_str}.slurm")
    # filepath for slurm stdout
    sbatch_out_fp = sbatch_dir.joinpath(sbatch_fp.name.replace(".slurm", "_%j.out"))
    sbatch_head = make_sbatch_head(**sbatch_head_kwargs)
    sbatch_regrid_kwargs = {
        "sbatch_fp": sbatch_fp,
        "sbatch_out_fp": sbatch_out_fp,
        "regrid_script": regrid_script,
        "regrid_dir": regrid_dir,
        "regrid_batch_fp": fp,
        "dst_fp": target_grid_fp,
        "no_clobber": False,
        "sbatch_head": sbatch_head,
    }
    write_sbatch_regrid(**sbatch_regrid_kwargs)
    sbatch_fps.append(sbatch_fp)

Remove existing slurm output files if desired:

In [5]:
_ = [fp.unlink() for fp in sbatch_dir.glob("*.out")]


Submit the jobs:

In [8]:
job_ids = [submit_sbatch(fp) for fp in sbatch_fps]
